In order to run this notebook, you will first need to run a dask scheduler and number of dask workers:
- Run a dask scheduler with:  ```dask-scheduler --scheduler-file=cluster.json```
- Run N dask workers with:  ```mpirun -np N dask-mpi --no-nanny --nthreads 10 --no-scheduler --scheduler-file cluster.json```

In [1]:
from dask.distributed import Client

In [2]:
# Run this if you are using an MPI-based cluster
client = Client(scheduler_file="cluster.json")

In [3]:
devs = [0, 1]
workers = list(client.has_what().keys())
worker_devs = workers[0:min(len(devs), len(workers))]

In [4]:
def set_visible(i, n):
    import os
    all_devices = list(range(n))
    vd = ",".join(map(str, all_devices[i:] + all_devices[:i]))
    print(str(vd))
    print("Selecting Device : "  + str(i))
    os.environ["CUDA_VISIBLE_DEVICES"] = vd
    
    import numba.cuda
    print("Cur device: " + str(numba.cuda.get_current_device().id))
    
[client.submit(set_visible, dev, len(devs), workers = [worker]) for dev, worker in zip(devs, worker_devs)]

[<Future: status: pending, key: set_visible-abcae3f11f146b3a6d9e8a6125e67de3>,
 <Future: status: pending, key: set_visible-3d1296012369cb47448939a5185d889c>]

In [5]:
import cudf
import dask_cudf
import numba.cuda
from dask_cuml.linear_regression import LinearRegression

In [6]:
import pandas as pd

X = cudf.DataFrame([('a', [0, 1, 2, 3, 4])])
y = cudf.Series([0, 1, 2, 3, 4])

In [7]:
X_df = dask_cudf.from_cudf(X, chunksize=1).persist()
y_df = dask_cudf.from_cudf(y, chunksize=1).persist()

Set each worker to host dfs on a different device. 

__Note__: You can ignore this if you started your workers with "CUDA_VISIBLE_DEVICE" already

In [8]:
lr = LinearRegression()

In [9]:
res = lr.fit(X_df, y_df)

res

In [10]:
lr.intercept_.result()

5

In [11]:
g = lr.predict(X_df)

In [12]:
print(str(g.result()))

      
0    1
1    2
2    3
3    4
4    5
